In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Deposit Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

3458b08be56503fb6fc5492089e2bdde3ac9bb20


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Time computations

In [9]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [10]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666155187000


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [11]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [12]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [13]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "d8f47d8d3ec5755b3904c04d33eeffceddb31e874b873009a32f4b9e9a140fa3"


#### The Counterparty

In [15]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [16]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [17]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "0f59b099b0c454f4ca97a5bb6746d024f5d1b0ad577b3ae99cdb024707d3b9c2"


## The Contract

We set the parameters for the ACTUS PAM contract.

In [18]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [19]:
yaml2json << EOI > deposit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat deposit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-18T00:00:00"}


Create the contract.

In [20]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  deposit.actus \
  --out-contract-file deposit-1.contract \
  --out-state-file /dev/null

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 10 minutes from now.

In [21]:
sed -i -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 10 * MINUTE))/" deposit-1.contract

View the contract.

In [22]:
json2yaml deposit-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1666155787000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

## Run the Contract

### Transaction 1. Create the ontract

Build the transaction.

In [23]:
TX_1=$(
marlowe create \
  --core-file deposit-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1


Sign the transaction.

In [24]:
cardano-cli transaction sign \
  --tx-body-file deposit-1.txbody \
  --out-file     deposit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [25]:
marlowe submit deposit-1.tx

"{\"blockHeaderHash\":\"780dde745ebfc5e1b92b3e8941b7c8cbbdd601fca9b3cd76aa8674d397d6c361\",\"blockNo\":284096,\"slotNo\":6152035}"


View the contract's UTxO.

In [26]:
CONTRACT_ADDR=$(marlowe-cli contract address)
echo "$CONTRACT_ADDR"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [27]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3fbccb83ec69f578dfc9dfddacfd1727a71108988ecf264daa2868f83a0590ea"


View the party's UTxOs.

In [28]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50     0        246785395 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [29]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0f59b099b0c454f4ca97a5bb6746d024f5d1b0ad577b3ae99cdb024707d3b9c2     1        250000000 lovelace + TxOutDatumNone


Watch the contract.

In [30]:
marlowe add "$CONTRACT_ID"

2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1


View the contract's history.

In [31]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50 (creation)
ContractId:      2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1
SlotNo:          6152035
BlockNo:         284096
BlockId:         780dde745ebfc5e1b92b3e8941b7c8cbbdd601fca9b3cd76aa8674d397d6c361
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [32]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$PARTY_ADDR" \
  --to-party "$PARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c


Sign the transaction.

In [33]:
cardano-cli transaction sign \
  --tx-body-file deposit-2.txbody \
  --out-file     deposit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [34]:
marlowe submit deposit-2.tx

"{\"blockHeaderHash\":\"350e5e444eb5dda57284b8ee30cc2db79370b0534569916bb5a119977ca7636b\",\"blockNo\":284097,\"slotNo\":6152046}"


View the contract's UTxO.

In [35]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_2//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "01a62f8a775798e6740e454035fabc0f10834e3fc94c17348fce2b4514b815d1"


View the party's UTxOs.

In [36]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     0        146031732 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [37]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0f59b099b0c454f4ca97a5bb6746d024f5d1b0ad577b3ae99cdb024707d3b9c2     1        250000000 lovelace + TxOutDatumNone
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     2        100000000 lovelace + TxOutDatumNone


View the contract's history.

In [38]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50 (creation)
ContractId:      2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1
SlotNo:          6152035
BlockNo:         284096
BlockId:         780dde745ebfc5e1b92b3e8941b7c8cbbdd601fca9b3cd76aa8674d397d6c361
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 3. Counterparty repays the loan's interest

The counterparty repays the interest.

In [39]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((INTEREST*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = 6f8b9755970498e3f93656a4c614473a74cf59765a9f7416e618ba9a1ece1407


Sign the transaction.

In [40]:
cardano-cli transaction sign \
  --tx-body-file deposit-3.txbody \
  --out-file     deposit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [41]:
marlowe submit deposit-3.tx

"{\"blockHeaderHash\":\"63ee8bbbac859e379bfd110faf8df41ddf0f0ea6d966df7e6eecb912c1efa685\",\"blockNo\":284098,\"slotNo\":6152069}"


View the contract's UTxO.

In [42]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_3//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6f8b9755970498e3f93656a4c614473a74cf59765a9f7416e618ba9a1ece1407     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "77455a0c5d2311a162edbbbf2d72ee4653586adaf05f83f58c0fcf065db0f2ef"


View the party's UTxOs.

In [43]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6f8b9755970498e3f93656a4c614473a74cf59765a9f7416e618ba9a1ece1407     2        2000000 lovelace + TxOutDatumNone
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     0        146031732 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [44]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6f8b9755970498e3f93656a4c614473a74cf59765a9f7416e618ba9a1ece1407     0        247308089 lovelace + TxOutDatumNone
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     2        100000000 lovelace + TxOutDatumNone


View the contract's history.

In [45]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50 (creation)
ContractId:      2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1
SlotNo:          6152035
BlockNo:         284096
BlockId:         780dde745ebfc5e1b92b3e8941b7c8cbbdd601fca9b3cd76aa8674d397d6c361
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 4. Counterparty repays the loan's principal

The counterparty repays the principal.

In [46]:
TX_4=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 079eef0a4384b159aedc754d65c3f4648f0b0d15cf457ef245fbcc360196e75e


Sign the transaction.

In [47]:
cardano-cli transaction sign \
  --tx-body-file deposit-4.txbody \
  --out-file     deposit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [48]:
marlowe submit deposit-4.tx

"{\"blockHeaderHash\":\"5d85da3e551a5d702242b4c7f670aceac6c6eafbbe78368995df4b429882df5b\",\"blockNo\":284099,\"slotNo\":6152081}"


Check that there is no UTxO for the contract.

In [49]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_4//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the party's UTxOs.

In [50]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
079eef0a4384b159aedc754d65c3f4648f0b0d15cf457ef245fbcc360196e75e     1        103000000 lovelace + TxOutDatumNone
6f8b9755970498e3f93656a4c614473a74cf59765a9f7416e618ba9a1ece1407     2        2000000 lovelace + TxOutDatumNone
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     0        146031732 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [51]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
079eef0a4384b159aedc754d65c3f4648f0b0d15cf457ef245fbcc360196e75e     0        146770238 lovelace + TxOutDatumNone
aae64d36fceb7e68fcdeb8ae601845edbe9198819ed88f717e801a7e0daae60c     2        100000000 lovelace + TxOutDatumNone


View the contract's history.

In [52]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50 (creation)
ContractId:      2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1
SlotNo:          6152035
BlockNo:         284096
BlockId:         780dde745ebfc5e1b92b3e8941b7c8cbbdd601fca9b3cd76aa8674d397d6c361
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Cleanup

Remove the contract from history tracking.

In [53]:
marlowe rm "$CONTRACT_ID"

2784ab25f7688295c9b5f69758068bb6a22fe2a3716af15a253b61e36f64fb50#1


In [54]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [55]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "eaa258587796ed96e0ec65784e6a9230c2255c8afedf421aac413d66bce05dd4"


In [56]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "557d39d4efc19a5d1b3dc51bae15fc84cd264b6df3f5a1a2bf1250fc8b32af6e"


Send the funds back to the faucet.

In [57]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "3ce4019d3b26efe353777b420a0726176c28121e502287ab54f68930d4ccb9c6"


See that the funds have been returned to the faucet.

In [58]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
